# Machine configuration example
This notebook describes the process of creating a machine configuration file for
use by FreeGSNKE.

A machine description in FreeGSNE is comprised of active coils
that can have their voltage modulated and act as controls for the plasma shape,
passive structures such as the tokamak wall and coil cans, and non-conducting
surfaces used to define the limit of the plasma.

In the following, we build up each of these elements into the structure required
by FreeGSNKE and save them for later use.

In [1]:
import numpy as np
import pickle

## Active coils
Here we define the characteristics and locations of the active coils of our
tokamak.

First, define the resistivity of the active coils.

In [2]:
eta_copper = 1.55e-8  # Resistivity in Ohm*m

Now write down the geometry of the coils. The below variables define the
locations of 5 coils: `P1_lower`, `P1_upper`, `P2_lower`, `P2_upper` and `Pz`.
Each coil has 2 windings.

We've also made a shortcut and assumed that `P1_lower` and `P1_upper` have the
same radial position and are equally distanced from the midplane in the vertical
direction. The same applies to `P2_lower` and `P2_upper`.

In [9]:
# P1_lower and P1_upper windings radial positions
P1_r = [1.0, 1.02]

# P1_upper windings vertical positions
P1_upper_z = [1.1, 1.12]

# P1_lower and P1_upper width and height
P1_dr = 0.02
P1_dz = 0.02

# As above, but for the P2_lower and P2_upper coils
P2_r = [1.75, 1.77]
P2_upper_z = [0.6, 0.62]
P2_dr = 0.02
P2_dz = 0.02

# Finally, the Pz coil
Pz_r = [1.0, 1.02]
Pz_z = [0.0, 0.0]
Pz_dr = 0.02
Pz_dz = 0.02

Next, we populate a dictionary with these values. The dictionary (here called
`active_coils_dict`) has a very particular format. Each entry to the dictionary
should be another dictionary describing a coil. This dictionary can take one of
two forms:  
1. One with the keys described below, or
2. One with labels corresponding to multiple coils.

This format allows for standalone coils or groups of coils to be defined. We
first demonstrate the standalone coil `Pz`, then show how `P1_upper` and
`P1_lower` can be grouped, and again for `P2_upper` and `P2_lower`.

The keys for the coil dictionaries are:
- `R`: list of radial positions of windings
- `Z`: list of vertical positions of windings
- `dR`: width
- `dZ`: height
- `resistivity`: resistivity in Ohm*m
- `polarity`: circuit wiring
- `multiplier`: current multiplier (only used for solenoid, see later)

First instantiate the dictionary so we can populate it incrementally

In [10]:
active_coils_dict = {}

Let's  define the standalone Pz coil first.

In [ ]:
active_coils_dict["Pz"] = {
    "R": Pz_r,
    "Z": Pz_z,
    "dR": Pz_dr,
    "dZ": Pz_dz,
    "resistivity": eta_copper,
    "polarity": 1,
}

Now let's define the `P1_upper` and `P1_lower` coils.

In [7]:
# The P1 coil group is instantiated as a dictionary
active_coils_dict["P1"] = {}

# Populate with the P1_upper coil information
active_coils_dict["P1"]["upper"] = {
    "R": P1_r,
    "Z": P1_upper_z,
    "dR": P1_dr,
    "dZ": P1_dz,
    "resistivity": eta_copper,
    "polarity": 1,
}

# Populate with the P1_lower coil information
active_coils_dict["P1"]["lower"] = {
    "R": P1_r,
    "Z": [-1*z for z in P1_upper_z],
    "dR": P1_dr,
    "dZ": P1_dz,
    "resistivity": eta_copper,
    "polarity": 1,
}

We follow the same procedure for `P2_upper` and `P2_lower`.

In [ ]:
active_coils_dict["P2"] = {}
active_coils_dict["P2"]["upper"] = {
    "R": P2_r,
    "Z": P2_upper_z,
    "dR": P2_dr,
    "dZ": P2_dz,
    "resistivity": eta_copper,
    "polarity": 1,
    "multiplier": 1
}
active_coils_dict["P2"]["lower"] = {
    "R": P2_r,
    "Z": [-1*z for z in P2_upper_z],
    "dR": P2_dr,
    "dZ": P2_dz,
    "resistivity": eta_copper,
    "polarity": 1,
    "multiplier": 1
}

Lastly for the active coils, we need to define a solenoid.

In [ ]:
active_coils_dict["Solenoid"] = {
    "R": [0.19475]*324,
    "Z": list(np.linspace(-1.5, 1.5, 324)),
    "dR": 0.012,
    "dZ": 0.018,
    "polarity": 1,
    "resistivity": eta_copper,
    "multiplier": 1
}

Now we can save the active coils dictionary for later use by FreeGSNKE.

In [11]:
with open("active_coils.pickle", "wb") as f:
    pickle.dump(active_coils_dict, f)